In [31]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [33]:
liste = []

a = 1
while a <= 50:

    URL="https://www.dr.com.tr/CokSatanlar/Kitap?_gl=1*3k3q8z*_up*MQ..&gclid=CjwKCAjwoa2xBhACEiwA1sb1BMlrKxhB-PSw3QfYDqYK4ALFGcajFOcLNuqeBInmcB9KaEepDJ1AshoC8nUQAvD_BwE&gclsrc=aw.ds&Page="+str(a)+""

    r=requests.get(URL)
    soup = BeautifulSoup(r.content,'html.parser')

    dokuman1 = soup.find("div",{"class":"facet__products-list js-facet-product-list"})
    dokuman2 = dokuman1.find_all("div",{"class":"product-img"})


    for detaylar in dokuman2:
        link_sonu = detaylar.a.get("href")
        link_bası = "https://www.dr.com.tr"
        link = link_bası+link_sonu

        r1 = requests.get(link)
        soup1 = BeautifulSoup(r1.content,"html.parser")


        fiyat = soup1.find("div",{"class":"salePrice"})
        yeni_fiyat = fiyat.find("span",{"class":"current-price js-text-current-price"}).text

        try:
            orijinal_fiyat = fiyat.find("span",{"class":"lowest-price"}).text
        except:
            orijinal_fiyat = yeni_fiyat

            
        try:
            oran = soup1.find("div",{"class":"lowestPrice"})
            indirim_oranı = oran.find("span",{"class":"lowest-rate"}).text
        except:
            indirim_oranı = ""



        try:
            degerlendirme1 = soup1.find("div",{"class":"prd-rate-title-wrapper mt-5 dr-flex-start js-scroll-comment scroll-trigger-comment"})
            degerlendirme2 = int((degerlendirme1.find("span").text.replace("(","").replace(")","")))
        except:
            degerlendirme2 = 0


        soup2 = soup1.find("div",{"class":"product-property"}).find_all("li")
        for detay in soup2:
            key = detay.find_all("strong")[0].text.strip().replace("'","").replace("'","")
            value = detay.find_all("span")[0].text.strip()

            if key == "Kitap Adı:":
                kitap_adı = value
            elif key == "Yazar:":
                yazar = value
            elif key == "Yayınevi:":
                yayınevi = value
            elif key == "Hamur Tipi:":
                hamur_tipi = value
            elif key == "Sayfa Sayısı:":
                try:
                    sayfa_sayısı = int(value)
                except:
                    sayfa_sayısı = 0
            elif key == "Ebat:":
                ebat = value
            elif key == "İlk Baskı Yılı:":
                ilk_baskı_yılı = int(value)
            elif key == "Baskı Sayısı:":
                baskı_sayısı = value
            elif key == "Dil:":
                dil = value
            elif key == "Barkod:":
                barkod = value
            else:
                ""


        foto1 = soup1.find("div",{"class":"swiper-wrapper js-container-detail-swiper"})
        foto2 = foto1.img.get("src")
        
        liste.append([barkod,kitap_adı,yazar,yayınevi,sayfa_sayısı,hamur_tipi,ebat,ilk_baskı_yılı,baskı_sayısı,degerlendirme2,dil,yeni_fiyat,orijinal_fiyat,indirim_oranı,foto2,link])
        
    a = a+1

In [34]:
dataFrame = pd.DataFrame(liste)
dataFrame.columns = ["barkod","ürün_adı","yazar","yayınevi","sayfa","hamur_tipi","ebat","ilk_baskı_yılı","baskı_sayısı","degerlendirme_sayısı","dil","yeni_fiyat","orijinal_fiyat","indirim_oranı","fotoğraf","link"]

In [36]:
dataFrame.to_excel("DR_Kitap_Veri.xlsx")